In [1]:
from gezi.common import *
sys.path.append('..')
gezi.set_pandas()
# gezi.set_pandas_widder()
from src.config import *
gezi.init_flags()

In [2]:
root = FLAGS.root

In [3]:
len(pd.read_csv(f'{root}/train_orders.csv'))

139256

In [4]:
def create_df(folder, workers=80):
  def _create_df(fpath):
    df = pd.read_json(fpath, dtype={'cell_type': 'category', 'source': 'str'}).reset_index().rename({"index":"cell_id"}, axis=1)
    df["id"] = fpath.rsplit(".", 1)[0].rsplit("/", 1)[-1]
    return df
  dfs = gezi.prun(_create_df, glob.glob(f'{folder}/*.json'), workers)
  df = pd.concat(dfs)
  df['source'] = df.source.apply(lambda x: x.replace('\n', BR))
  return df

In [6]:
workers = 80
train_file = f'../working/train.fea'
if os.path.exists(train_file):
  df = pd.read_feather(train_file)
else:
  df = create_df(f'../working/train.fea', workers)

In [9]:
df.columns

Index(['cell_id', 'cell_type', 'source', 'id', 'n_words', 'cid', 'ancestor_id',
       'parent_id', 'n_cell', 'n_code_cell', 'n_markdown_cell',
       'markdown_frac', 'rank', 'code_rank', 'markdown_rank', 'rel_rank',
       'pct_rank', 'fold', 'worker'],
      dtype='object')

In [14]:
df0 = df[df.fold==0]

In [15]:
len(df0.id.unique())

27590

In [18]:
len(df0[df0.parent_id.isnull()].id.unique())

23690

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-small')
if not 'input_ids' in df.columns:
  if tokenizer.convert_tokens_to_ids(BR) == tokenizer.unk_token_id:
    assert len(BR) == 1
    tokenizer.add_tokens([BR], special_tokens=False)
  input_ids_list = gezi.prun(lambda x: tokenizer(x).input_ids, df.source.values, 80, desc='tokenize')
  df['input_ids'] = input_ids_list
  df['tokens'] = gezi.prun(tokenizer.convert_ids_to_tokens, df.input_ids.values, 80, desc='convert_ids_to_tokens')
  df.reset_index().to_feather(train_file)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


run:   0%|          | 0/79634 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

Exception ignored in: <function _releaseLock at 0x7fee04acf7a0>
Traceback (most recent call last):
  File "/home/huigecheng/envs/pku/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt
Exception ignored in: <function _releaseLock at 0x7fee04acf7a0>
Traceback (most recent call last):
  File "/home/huigecheng/envs/pku/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():

run:   0%|          | 0/79633 [00:00<?, ?it/s]

Exception ignored in: <function _releaseLock at 0x7fee04acf7a0>
Traceback (most recent call last):
  File "/home/huigecheng/envs/pku/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():

run:   0%|          | 0/79633 [00:00<?, ?it/s]

run:   0%|          | 0/79634 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

In [ ]:
df

,cell_id,cell_type,source,id,input_ids,tokens
0,1c7a5a71,code,"import warningsʶwarnings.filterwarnings(""ignore"")ʶʶimport sqlite3ʶimport pandas as pdʶimport numpy as npʶimport nltkʶimport stringʶimport matplotlib.pyplot as pltʶimport seaborn as snsʶfrom sklearn.feature_extraction.text import TfidfTransformerʶfrom sklearn.feature_extraction.text import TfidfVectorizerʶʶfrom sklearn.feature_extraction.text import CountVectorizerʶfrom sklearn.metrics import confusion_matrixʶfrom sklearn import metricsʶfrom sklearn.metrics import roc_curve, aucʶfrom nltk.ste...",a3baf6134c8506,"[1, 6306, 11917, 128001, 11917, 260, 28865, 60656, 268, 555, 309, 66478, 309, 285, 128001, 128001, 6306, 120474, 508, 128001, 6306, 67927, 283, 845, 407, 128001, 6306, 36221, 11751, 283, 76767, 128001, 6306, 90668, 39501, 128001, 6306, 4022, 128001, 6306, 8358, 33918, 14434, 260, 11751, 33918, 283, 28944, 297, 128001, 6306, 2164, 6107, 283, 41339, 268, 128001, 292, 33566, 29274, 260, 51532, 616, 113492, 260, 12948, 6306, 897, 59426, 1892, 86911, 649, 128001, 292, 33566, 29274, 260, 51532, 61...","[[CLS], ▁import, ▁warnings, ʶ, ▁warnings, ., filter, warning, s, (, "", ignore, "", ), ʶ, ʶ, ▁import, ▁sqlite, 3, ʶ, ▁import, ▁pandas, ▁as, ▁p, d, ʶ, ▁import, ▁num, py, ▁as, ▁np, ʶ, ▁import, ▁nl, tk, ʶ, ▁import, ▁string, ʶ, ▁import, ▁mat, plot, lib, ., py, plot, ▁as, ▁pl, t, ʶ, ▁import, ▁sea, born, ▁as, ▁sn, s, ʶ, ▁from, ▁sk, learn, ., feature, _, extraction, ., text, ▁import, ▁T, fid, f, Transform, er, ʶ, ▁from, ▁sk, learn, ., feature, _, extraction, ., text, ▁import, ▁T, fid, f, Vector, izer..."
1,68f71f96,code,"# using the SQLite Table to read data.ʶcon = sqlite3.connect('../input/database.sqlite')ʶ#con = sqlite3.connect('database.sqlite') ʶʶ#filtering only positive and negative reviews i.e. ʶ# not taking into consideration those reviews with Score=3ʶfiltered_data = pd.read_sql_query(""""""SELECT * FROM Reviews WHERE Score != 3 LIMIT 100000"""""", con) ʶʶ# Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating.ʶdef partition(x):ʶ if x < 3:ʶ return 0ʶ return...",a3baf6134c8506,"[1, 953, 478, 262, 78550, 3751, 264, 623, 514, 260, 128001, 4636, 1842, 120474, 508, 260, 30407, 555, 280, 260, 260, 320, 42177, 320, 54698, 260, 51972, 5936, 280, 285, 128001, 953, 5699, 1842, 120474, 508, 260, 30407, 555, 280, 54698, 260, 51972, 5936, 280, 285, 128001, 128001, 953, 28865, 510, 364, 1453, 263, 2330, 1937, 584, 260, 473, 260, 128001, 953, 298, 787, 352, 3937, 421, 1937, 275, 13938, 1510, 508, 128001, 16334, 616, 9832, 1842, 845, 407, 260, 8523, 616, 51972, 616, 47975, 555, 3...","[[CLS], ▁#, ▁using, ▁the, ▁SQLite, ▁Table, ▁to, ▁read, ▁data, ., ʶ, ▁con, ▁=, ▁sqlite, 3, ., connect, (, ', ., ., /, input, /, database, ., sql, ite, ', ), ʶ, ▁#, con, ▁=, ▁sqlite, 3, ., connect, (, ', database, ., sql, ite, ', ), ʶ, ʶ, ▁#, filter, ing, ▁only, ▁positive, ▁and, ▁negative, ▁reviews, ▁i, ., e, ., ʶ, ▁#, ▁not, ▁taking, ▁into, ▁consideration, ▁those, ▁reviews, ▁with, ▁Score, =, 3, ʶ, ▁filtered, _, data, ▁=, ▁p, d, ., read, _, sql, _, query, (, "", "", "", SELECT, ▁*, ▁FROM, ▁Reviews..."
2,f0a7f5c4,code,"display = pd.read_sql_query("""""" SELECT UserId, ProductId, ProfileName, Time, Score, Text, COUNT(*) FROM Reviews GROUP BY UserId HAVING COUNT(*)>1 """""", con)ʶprint(display.shape)ʶdisplay.head()ʶ",a3baf6134c8506,"[1, 1689, 1842, 845, 407, 260, 8523, 616, 51972, 616, 47975, 555, 309, 309, 309, 45786, 5675, 29935, 261, 4899, 29935, 261, 12028, 15303, 261, 2210, 261, 13938, 261, 7655, 261, 73857, 555, 1225, 285, 11371, 8939, 30686, 9506, 5675, 29935, 92773, 73857, 555, 1225, 285, 1504, 435, 307, 309, 309, 261, 4636, 285, 128001, 2118, 555, 35459, 260, 29753, 285, 128001, 1689, 260, 5563, 555, 285, 128001, 2]","[[CLS], ▁display, ▁=, ▁p, d, ., read, _, sql, _, query, (, "", "", "", ▁SELECT, ▁User, Id, ,, ▁Product, Id, ,, ▁Profile, Name, ,, ▁Time, ,, ▁Score, ,, ▁Text, ,, ▁COUNT, (, *, ), ▁FROM, ▁Reviews, ▁GROUP, ▁BY, ▁User, Id, ▁HAVING, ▁COUNT, (, *, ), 

In [9]:
tokenizer.vocab_size

128000

In [8]:
from gensim.models import Word2Vec

In [16]:
# 10w using 3.14min 1 epoch for emb 256
# for emb 128 only 0.3min
def gen_w2v(df, tokenizer, name='tokens', window=16, min_count=5, emb_dim=256, limit=0):
  sentences = df[name].values
  ic(len(sentences))

  if limit:
    sentences = sentences[:limit]
    ic(len(sentences))
    name = name + f'.limit{limit}'
  monitor = gezi.MonitorCallback(name) 
  w2v = Word2Vec(sentences, vector_size=emb_dim, window=window, min_count=min_count, sg=1, workers=cpu_count(), epochs=10, callbacks=[monitor])
  
  root = f'../input/w2v/{emb_dim}'
  ofile = f'{root}/{name}.pkl'
  gezi.try_mkdir(os.path.dirname(ofile))
  gezi.save(w2v, ofile)
  emb = np.random.uniform(-0.05, 0.05,(tokenizer.vocab_size, emb_dim))
  for i in range(tokenizer.vocab_size):
    token = tokenizer.convert_ids_to_tokens(i)
    if token in w2v.wv:
      emb[i] = w2v.wv[token]
  ofile = f'{root}/{name}.npy'
  np.save(ofile, emb)
  return w2v

In [18]:
gen_w2v(df, tokenizer, emb_dim=128)

[06/23/22 19:06:50] 3754286411.py:4 in gen_w2v()
                    len(sentences): 6370646


KeyboardInterrupt: 

In [ ]:
df_cells = df.groupby('id')['cell_id'].count().reset_index(name='n_cell_id')
df_cells

,id,n_cell_id
0,00001756c60be8,58
1,00015c83e2717b,93
2,0001bdd4021779,13
3,0001daf4c2c76d,229
4,0002115f48f982,9
...,...,...
139251,fffc30d5a0bc46,31
139252,fffc3b44869198,24
139253,fffc63ff750064,26
139254,fffcd063cda949,36


In [ ]:
df_orders = pd.read_csv(f'{root}/train_orders.csv')
df_orders

,id,cell_order
0,00001756c60be8,1862f0a6 448eb224 2a9e43d6 7e2f170a 038b763d 77e56113 2eefe0ef 1ae087ab 0beab1cd 8ffe0b25 9a78ab76 0d136e08 8a4c95d1 23705731 ebe125d5 aaad8355 d9dced8b 21616367 86497fe1 c3ce0945 e2c8e725 a6357f7e ff7c44ed ac301a84 0e7c906e dd0c804a 45082c89 781bbf3c 4bb2e30a bd94f005 63c26fa2 62638fba 3e5f860d bb69e88c 6b5664c7 3eebeb87 23783525 36002912 bfbde93e 8522781a 1496beaf 8ca8392c b69a4f9b 17ec3fc4 503926eb 76512d50 032e2820 a98c5d9f 06365725 8554b284 59959af5 2e1a5949 80151ab7 fcb6792d 5bf9ca51 9...
1,00015c83e2717b,2e94bd7a 3e99dee9 b5e286ea da4f7550 c417225b 51e3cd89 2600b4eb 75b65993 cf195f8b 25699d02 72b3201a f2c750d3 de148b56 42749e24 fbe3e811 9901472c 10377ef8 1f462e2f fceeb3e6 ceba8ae0 924c9f0f 2af2a41a 91e68f13 9216a113 63753f10 d5aee1e4 dc8a39a5 3d0a28c2 657a8804 0eea9701 a8fcc3e3 a6f8f9f1 7223cfc2 a166703b df6b3ccb da8b817a a9b266d2 41beeead 3e17f424 cd61f6d1 42f0c365 cc8d23d8 be6c9079 ad42abc1 7894c4e8 eab2b130 cc1add42 16b0d436 a3e791de 183226e4 02ef0932 03441163 f2915b9f d429a743 4d5ebd46 3...
2,0001bdd4021779,3fdc37be 073782ca 8ea7263c 80543cd8 38310c80 073e27e5 015d52a4 ad7679ef 7fde4f04 07c52510 0a1a7a39 0bcd3fef 58bf360b
3,0001daf4c2c76d,97266564 a898e555 86605076 76cc2642 ef279279 df6c939f 2476da96 00f87d0a ae93e8e6 58aadb1d d20b0094 986fd4f1 b4ff1015 9b761026 6f271c86 97c3f99b 2451daed cfa510c5 374a5179 df5f7c1f 27060ab9 4c7395ec 2d2f9893 35c2812a a6617a52 cd2cf0ad 50cc03dd a57f19d3 4843d706 09c20343 4385fc90 42241549 edaaeecd 1ed4aebf 54ddeb88 08dac4fd 3307d698 ebb2c2f7 ac2b6b85 34f430ca e54d5869 39c98d5f d6c5745e 966d9c8a 27f52431 4524758d 91a4584a 4ef89e46 82d7d8ca b6c234fb 2d0e1fda 434b8e0c bd0b9dd7 7cececae a3edb582 a...
4,0002115f48f982,9ec225f0 18281c6c e3b6b115 4a044c54 365fe576 a3188e54 b3f6e12d ee7655ca 84125b7a
...,...,...
139251,fffc30d5a0bc46,09727c0c ff1ea6a0 ddfef603 a01ce9b3 3ba953ee bf92a015 f4a0492a 095812e6 53125cfe aa32a700 63340e73 06d8c952 400ceb37 55fa544e d3647ac8 f8e7d47e 7102897a a3a47643 ebd32b52 53caf4d9 b314d9e3 2986b5e1 7eb7350d 681c0d9b f083e1de 704b3ee9 a8e79846 eb887c6c 4faf1ad7 6122a91f 015cff34
139252,fffc3b44869198,978a5137 faa48f03 28dfb12a eea2e812 64fef97c 4e0d1510 58e68f2c 8784e700 4bd5a4cf dc14bfec 2aff7603 8047ded2 ac536a5b d2ec3efb d7172db2 0c5441b4 8bb4abd2 aa168c04 b01422ce 6e67b343 40e930ff b1873cbb 76e0f2a7 233d93b9
139253,fffc63ff750064,5015c300 411b85d9 8238198c f4781d1d b5532930 e1f223e5 e7e67119 4aaf741d 7229cce6 a7fa3628 e4c2fa86 1f8f9d14 7a52357a e9b6265c b32dc5d2 41ccef58 6aff1ae7 29e1f39d b3c6bc16 37cbd460 79e4e69f ee9b8d31 8b54cf58 deead32c 56ee6e4c 56aa8da7
139254,fffcd063cda949,7e6266ad d8281fc5 d4ffcaef 3e0e4a47 21387fc8 cc229f9a baed9c8b d1bb21aa 82979992 65f95dad eba4fa9e c97e268d 7e1a6588 055e0d2e c090bd72 0bf4ab17 26374693 5098ff9a ef42f19d 28f8bc15 b47bacee 0fad41c8 4f671884 4b254efa 74802b2f 7c33c5b2 7168afec 9a2cdb85 fb7456dd 1fd0781b ff949d21 a16411ac 7a1b4718 777b3fc0 f90ee056 83dce89a


In [ ]:
df_orders['n_cell_id'] = df_orders['cell_order'].apply(lambda x: len(x.split()))

In [ ]:
df_orders.describe([.25,.5,.75,.9,.95,.99])

,n_cell_id
count,139256.0000
mean,45.7477
std,40.0858
min,2.0000
25%,21.0000
50%,35.0000
75%,57.0000
90%,90.0000
95%,118.0000
99%,194.4500


In [ ]:
df_cells.describe([.25,.5,.75,.9,.95,.99])

,n_cell_id
count,139256.0000
mean,45.7477
std,40.0858
min,2.0000
25%,21.0000
50%,35.0000
75%,57.0000
90%,90.0000
95%,118.0000
99%,194.4500


In [ ]:
df[df.id=='fffc30d5a0bc46']

,index,cell_id,cell_type,source,id,input_ids
1004739,0,ff1ea6a0,code,import warningsʶwarnings.filterwarnings('ignore')ʶʶimport numpy as npʶimport pandas as pdʶʶimport seaborn as snsʶfrom matplotlib import pyplot as pltʶ%matplotlib inlineʶʶimport sklearnʶfrom sklearn.model_selection import train_test_splitʶfrom sklearn.metrics import accuracy_scoreʶfrom sklearn.neighbors import KNeighborsClassifierʶfrom sklearn.linear_model import LinearRegressionʶʶimport tensorflow as tfʶfrom tensorflow import kerasʶʶfrom sklearn import preprocessingʶtf.__version__,fffc30d5a0bc46,"[1, 6306, 11917, 128001, 11917, 260, 28865, 60656, 268, 555, 280, 66478, 280, 285, 128001, 128001, 6306, 36221, 11751, 283, 76767, 128001, 6306, 67927, 283, 845, 407, 128001, 128001, 6306, 2164, 6107, 283, 41339, 268, 128001, 292, 8358, 33918, 14434, 6306, 35512, 33918, 283, 28944, 297, 128001, 4639, 13261, 33918, 14434, 30426, 128001, 128001, 6306, 33566, 29274, 128001, 292, 33566, 29274, 260, 19928, 616, 57907, 6306, 2184, 616, 9982, 616, 43483, 128001, 292, 33566, 29274, 260, 56282, 6306,..."
1004740,1,a01ce9b3,code,weather_df = pd.read_csv('../input/weatherHistory.csv'),fffc30d5a0bc46,"[1, 1742, 616, 32392, 1842, 845, 407, 260, 8523, 616, 76413, 555, 280, 260, 260, 320, 42177, 320, 22399, 35874, 260, 76413, 280, 285, 2]"
1004741,2,bf92a015,code,weather_df.head(),fffc30d5a0bc46,"[1, 1742, 616, 32392, 260, 5563, 555, 285, 2]"
1004742,3,095812e6,code,weather_df.columnsʶweather_df.shapeʶweather_df.describe()ʶweather_df.info()ʶweather_df.isnull().any()ʶweather_df.isnull().all(),fffc30d5a0bc46,"[1, 1742, 616, 32392, 260, 39163, 268, 128001, 1742, 616, 32392, 260, 29753, 128001, 1742, 616, 32392, 260, 118314, 555, 285, 128001, 1742, 616, 32392, 260, 10101, 555, 285, 128001, 1742, 616, 32392, 260, 1890, 43858, 555, 285, 260, 11771, 555, 285, 128001, 1742, 616, 32392, 260, 1890, 43858, 555, 285, 260, 3068, 555, 285, 2]"
1004743,4,aa32a700,code,"round(100*(weather_df.isnull().sum()/len(weather_df.index)),2)",fffc30d5a0bc46,"[1, 1489, 555, 2273, 1225, 555, 22399, 616, 32392, 260, 1890, 43858, 555, 285, 260, 17608, 555, 285, 320, 12630, 555, 22399, 616, 32392, 260, 13915, 285, 285, 261, 445, 285, 2]"
1004744,5,06d8c952,code,weather_df['Precip Type'].value_counts(),fffc30d5a0bc46,"[1, 1742, 616, 32392, 2550, 280, 17108, 31100, 5004, 280, 592, 260, 11651, 616, 21036, 268, 555, 285, 2]"
1004745,6,400ceb37,code,"weather_df.loc[weather_df['Precip Type'].isnull(),'Precip Type']='rain'",fffc30d5a0bc46,"[1, 1742, 616, 32392, 260, 28372, 2550, 22399, 616, 32392, 2550, 280, 17108, 31100, 5004, 280, 592, 260, 1890, 43858, 555, 285, 261, 280, 17108, 31100, 5004, 280, 592, 1510, 280, 26790, 280, 2]"
1004746,7,55fa544e,code,"round(100*(weather_df.isnull().sum()/len(weather_df.index)),2)",fffc30d5a0bc46,"[1, 1489, 555, 2273, 1225, 555, 22399, 616, 32392, 260, 1890, 43858, 555, 285, 260, 17608, 555, 285, 320, 12630, 555, 22399, 616, 32392, 260, 13915, 285, 285, 261, 445, 285, 2]"
1004747,8,f8e7d47e,code,"plt.scatter(x = 'Wind Speed (km/h)',y = 'Wind Bearing (degrees)',data=weather_df)",fffc30d5a0bc46,"[1, 28944, 297, 260, 69916, 555, 982, 1842, 382, 50004, 7628, 287, 6275, 320, 1537, 285, 280, 261, 608, 1842, 382, 50004, 24029, 287, 72511, 285, 280, 261, 9832, 1510, 22399, 616, 32392, 285, 2]"
1004748,9,7102897a,code,"plt.figure(figsize = (10,10))ʶplt.subplot()ʶplt.subplot(2,3,1)ʶplt.title('Wind Speed (km/h)')ʶplt.hist(x = 'Wind Speed (km/h)',bins =20,data = weather_df)ʶʶplt.subplot(2,3,2)ʶplt.title('Apparent Temperature (C)')ʶplt.hist(x = 'Apparent Temperature (C)',bins =20,data = weather_df)ʶʶplt.subplot(2,3,3)ʶplt.title('Humidity')ʶplt.hist(x = 'Humidity',bins =20,data = weather_df)ʶʶplt.subplot(2,3,4)ʶplt.title('Wind Bearing (degrees)')ʶplt.hist(x = 'Wind Bearing (degrees)',bins =100,data = weather_df...",fffc30d5a0bc46,"[1, 28944, 297, 260, 22971, 555, 30134, 7283, 1842, 287, 894, 261, 894, 285, 285, 128001, 28944, 297, 260, 12109, 33918, 555, 285, 128001, 28944, 297, 260, 12109, 33918

In [ ]:
df['input_ids2'] = df['input_ids'].apply(lambda x: x[:128])
dfg = df.groupby('id')['input_ids2'].apply(lambda l: itertools.chain(*l)).reset_index(name='input_ids')
dfg['input_ids'] = dfg['input_ids'].apply(lambda x: list(x))
dfg['n_words'] = dfg.input_ids.apply(lambda x: len(list(x)))
dfg.describe(PERCENTILES)

,n_words
count,139256.0000
mean,2546.2496
std,2262.7916
min,12.0000
25%,1175.0000
50%,1930.0000
75%,3167.0000
90%,5015.0000
95%,6574.0000
99%,10994.7000


In [ ]:
df_code['input_ids2'] = df_code['input_ids'].apply(lambda x: x[:128])
dfg_code = df_code.groupby('id')['input_ids2'].apply(lambda l: itertools.chain(*l)).reset_index(name='input_ids')
dfg_code['input_ids'] = dfg_code['input_ids'].apply(lambda x: list(x))
dfg_code['n_words'] = dfg_code.input_ids.apply(lambda x: len(list(x)))
dfg_code.describe(PERCENTILES)

,n_words
count,139256.0000
mean,1936.9331
std,1683.2294
min,4.0000
25%,903.0000
50%,1501.0000
75%,2433.0000
90%,3772.0000
95%,4894.0000
99%,8175.4500


In [ ]:
df_markdown['input_ids2'] = df_markdown['input_ids'].apply(lambda x: x[:128])
dfg_markdown = df_markdown.groupby('id')['input_ids2'].apply(lambda l: itertools.chain(*l)).reset_index(name='input_ids')
dfg_markdown['input_ids'] = dfg_markdown['input_ids'].apply(lambda x: list(x))
dfg_markdown['n_words'] = dfg_markdown.input_ids.apply(lambda x: len(list(x)))
dfg_markdown.describe(PERCENTILES)

,n_words
count,139256.0000
mean,609.3166
std,875.0669
min,3.0000
25%,115.0000
50%,312.0000
75%,771.0000
90%,1507.0000
95%,2157.0000
99%,4026.0000


In [ ]:
PERCENTILES = [.25,.5,.75,.9,.95,.99]

In [ ]:
df['n_words'] = df.input_ids.apply(len)
df.describe(PERCENTILES)

,index,n_words
count,6370646.0000,6370646.0000
mean,39.9360,108.0791
std,49.0379,1746.4885
min,0.0000,3.0000
25%,11.0000,16.0000
50%,26.0000,40.0000
75%,51.0000,97.0000
90%,89.0000,213.0000
95%,124.0000,326.0000
99%,231.0000,813.0000


In [ ]:
df_code = df[df.cell_type == 'code']
df_code.describe(PERCENTILES)

,index,n_words
count,4204582.0000,4204582.0000
mean,26.6307,117.2286
std,33.6353,886.3052
min,0.0000,3.0000
25%,7.0000,23.0000
50%,17.0000,52.0000
75%,34.0000,121.0000
90%,60.0000,253.0000
95%,83.0000,380.0000
99%,157.0000,941.0000


In [ ]:
df_markdown = df[df.cell_type == 'markdown']
df_markdown.describe(PERCENTILES)

,index,n_words
count,2166064.0000,2166064.0000
mean,65.7631,90.3189
std,62.1758,2728.6927
min,1.0000,3.0000
25%,29.0000,10.0000
50%,48.0000,21.0000
75%,82.0000,55.0000
90%,129.0000,124.0000
95%,171.0000,203.0000
99%,309.0000,505.0000


In [ ]:
df_code.groupby('id')['cell_id'].count().reset_index(name='n_cell_id')

,id,n_cell_id
0,00001756c60be8,30
1,00015c83e2717b,72
2,0001bdd4021779,11
3,0001daf4c2c76d,178
4,0002115f48f982,8
...,...,...
139251,fffc30d5a0bc46,22
139252,fffc3b44869198,23
139253,fffc63ff750064,22
139254,fffcd063cda949,24


In [ ]:
df_code.groupby('id')['cell_id'].count().reset_index(name='n_cell_id').describe(PERCENTILES)

,n_cell_id
count,139256.0000
mean,30.1932
std,26.9575
min,1.0000
25%,14.0000
50%,23.0000
75%,38.0000
90%,60.0000
95%,78.0000
99%,130.0000


In [ ]:
df_markdown.groupby('id')['cell_id'].count().reset_index(name='n_cell_id')

,id,n_cell_id
0,00001756c60be8,28
1,00015c83e2717b,21
2,0001bdd4021779,2
3,0001daf4c2c76d,51
4,0002115f48f982,1
...,...,...
139251,fffc30d5a0bc46,9
139252,fffc3b44869198,1
139253,fffc63ff750064,4
139254,fffcd063cda949,12


In [ ]:
df_markdown.groupby('id')['cell_id'].count().reset_index(name='n_cell_id').describe(PERCENTILES)

,n_cell_id
count,139256.0000
mean,15.5545
std,17.2238
min,1.0000
25%,5.0000
50%,11.0000
75%,20.0000
90%,34.0000
95%,47.0000
99%,81.0000


In [ ]:
dfg

,id,input_ids,n_words
0,00001756c60be8,"[1, 953, 329, 11369, 404, 1192, 699, 275, 386, 2136, 7027, 7296, 2627, 128001, 953, 325, 269, 3034, 293, 262, 4219, 89061, 320, 76502, 26224, 1115, 294, 3597, 294, 320, 320, 45552, 260, 549, 320, 1165, 89061, 320, 25805, 649, 271, 76502, 128001, 953, 434, 738, 261, 422, 280, 268, 656, 2136, 4507, 264, 2929, 128001, 128001, 6306, 36221, 11751, 283, 76767, 953, 7464, 16373, 128001, 6306, 67927, 283, 845, 407, 953, 514, 2466, 261, 33379, 1092, 273, 320, 1702, 287, 473, 260, 948, 260, 845, 407, ...",4951
1,00015c83e2717b,"[1, 6306, 36221, 11751, 283, 76767, 953, 7464, 16373, 128001, 6306, 67927, 283, 845, 407, 128001, 845, 407, 260, 6207, 616, 37896, 555, 309, 35459, 260, 10537, 616, 50585, 309, 261, 7369, 285, 128001, 6306, 2673, 268, 128001, 2118, 555, 3603, 260, 8375, 20837, 555, 309, 260, 260, 320, 42177, 309, 285, 285, 128001, 6306, 37821, 445, 128001, 6306, 74883, 128001, 6306, 8358, 33918, 14434, 260, 11751, 33918, 283, 28944, 297, 128001, 4639, 13261, 33918, 14434, 30426, 128001, 28944, 297, 260, 1963...",11879
2,0001bdd4021779,"[1, 6306, 67927, 283, 845, 407, 128001, 6306, 36221, 11751, 283, 76767, 128001, 6306, 8358, 33918, 14434, 260, 11751, 33918, 283, 28944, 297, 128001, 6306, 2164, 6107, 283, 41339, 268, 128001, 6306, 11917, 283, 2976, 268, 128001, 2976, 268, 260, 28865, 60656, 268, 555, 309, 66478, 309, 285, 2, 1, 64143, 1842, 845, 407, 260, 8523, 616, 76413, 555, 309, 320, 1165, 89061, 320, 42177, 320, 1867, 271, 41524, 271, 42855, 268, 320, 42855, 268, 260, 76413, 309, 285, 2, 1, 64143, 260, 5563, 555, 285,...",438
3,0001daf4c2c76d,"[1, 953, 329, 11369, 404, 1192, 699, 275, 386, 2136, 7027, 7296, 2627, 128001, 953, 325, 269, 3034, 293, 262, 4219, 89061, 320, 76502, 26224, 1115, 294, 3597, 294, 320, 320, 45552, 260, 549, 320, 1165, 89061, 320, 25805, 649, 271, 76502, 128001, 953, 434, 738, 261, 422, 280, 268, 656, 2136, 4507, 264, 2929, 128001, 128001, 6306, 36221, 11751, 283, 76767, 953, 7464, 16373, 128001, 6306, 67927, 283, 845, 407, 953, 514, 2466, 261, 33379, 1092, 273, 320, 1702, 287, 473, 260, 948, 260, 845, 407, ...",23333
4,0002115f48f982,"[1, 6306, 36221, 11751, 283, 76767, 953, 7464, 16373, 128001, 6306, 67927, 283, 845, 407, 953, 514, 2466, 261, 33379, 1092, 273, 320, 1702, 287, 473, 260, 948, 260, 845, 407, 260, 8523, 616, 76413, 285, 128001, 6306, 8358, 33918, 14434, 260, 11751, 33918, 283, 28944, 297, 128001, 6306, 2164, 6107, 283, 41339, 268, 128001, 4639, 13261, 33918, 14434, 30426, 128001, 6306, 2673, 268, 128001, 2118, 555, 3603, 260, 8375, 20837, 555, 309, 260, 260, 320, 42177, 309, 285, 285, 128001, 2, 1, 64143, 18...",370
...,...,...,...
139251,fffc30d5a0bc46,"[1, 6306, 11917, 128001, 11917, 260, 28865, 60656, 268, 555, 280, 66478, 280, 285, 128001, 128001, 6306, 36221, 11751, 283, 76767, 128001, 6306, 67927, 283, 845, 407, 128001, 128001, 6306, 2164, 6107, 283, 41339, 268, 128001, 292, 8358, 33918, 14434, 6306, 35512, 33918, 283, 28944, 297, 128001, 4639, 13261, 33918, 14434, 30426, 128001, 128001, 6306, 33566, 29274, 128001, 292, 33566, 29274, 260, 19928, 616, 57907, 6306, 2184, 616, 9982, 616, 43483, 128001, 292, 33566, 29274, 260, 56282, 6306,...",1225
139252,fffc3b44869198,"[1, 6306, 8358, 33918, 14434, 260, 11751, 33918, 283, 28944, 297, 128001, 6306, 36221, 11751, 283, 76767, 128001, 6306, 14371, 11751, 260, 68558, 5879, 283, 5515, 128001, 6306, 33566, 29274, 260, 25016, 616, 19928, 128001, 6306, 33566, 29274, 260, 19928, 616, 57907, 128001, 6306, 67927, 283, 845, 407, 2, 1, 2184, 1842, 845, 407, 260, 8523, 616, 76413, 555, 280, 260, 260, 320, 42177, 320, 1537, 5183, 271, 1764, 1406, 271, 21727, 320, 13613, 616, 23822, 260, 76413, 280, 285, 2, 1, 2184, 616, 9...",1272
139253,fffc63ff750064,"[1, 6306, 36221, 11751, 283, 76767, 128001, 6306, 67927, 283, 845, 407, 128001, 6306, 8358, 33918, 14434, 260, 11751, 33918, 283, 28944, 297, 128001, 6306, 2164, 6107, 283, 41339, 268, 128001, 128001, 292, 33566, 29274, 6306, 10125, 128001, 292, 33566, 29274, 2

In [ ]:
dfg.describe([.25,.5,.75,.9,.95,.99])

,n_words
count,139256.0000
mean,4944.3743
std,14845.9758
min,12.0000
25%,1654.0000
50%,2953.0000
75%,5312.0000
90%,9113.0000
95%,12622.5000
99%,27145.1000


In [ ]:
df_test = create_df(f'{root}/test', 1)

run:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df_test

,cell_id,cell_type,source,id
0,ddfd239c,code,"import numpy as np # linear algebraʶimport pandas as pd # data processing,ʶimport matplotlib.pyplot as pltʶfrom sklearn.decomposition import PCAʶfrom sklearn.preprocessing import StandardScalerʶfrom sklearn.preprocessing import scaleʶfrom sklearn.impute import SimpleImputerʶʶʶimport osʶfor dirname, _, filenames in os.walk('/kaggle/input'):ʶ for filename in filenames:ʶ print(os.path.join(dirname, filename))",0009d135ece78d
1,c6cd22db,code,df = pd.read_csv('/kaggle/input/breast-cancer-wisconsin-data/data.csv')ʶdf,0009d135ece78d
2,1372ae9b,code,"numerical_data = df.loc[:, ~df.columns.isin(['id', ""diagnosis""])]ʶʶlabels = df[""diagnosis""].factorize(['B','M'])[0]ʶʶheader_labels = pd.DataFrame(data=labels, columns=[""diagnosis""])",0009d135ece78d
3,90ed07ab,code,"def comparison_plot_maker(data_1, data_2, name, column_name_1, column_name_2):ʶ # Scaling Data for testingʶ # data_1 = scale(data_1)ʶ # data_2 = scale(data_2)ʶʶ range = np.random.randn(len(data_1))ʶ plt.scatter(range, data_1, label=column_name_1, color='orange')ʶ plt.scatter(range, data_2, label=column_name_2, color='green')ʶ plt.title(name)ʶ plt.xlabel('X-Axis')ʶ plt.ylabel('Y-Axis')ʶ plt.legend()ʶ plt.show()ʶ",0009d135ece78d
4,7f388a41,code,"# Ploting data with different columnsʶ#####################################ʶcomparison_plot_maker(numerical_data[""radius_mean""], numerical_data[""radius_worst""], ""Mean Radius vs Worst Radius"", ""Mean Radius"", ""Worst Radius"")ʶcomparison_plot_maker(numerical_data[""perimeter_se""], numerical_data[""perimeter_worst""], ""S.D Perimeter vs Worst Perimeter"", ""S.D Perimeter"", ""Worst Perimeter"")ʶcomparison_plot_maker(numerical_data[""compactness_mean""], numerical_data[""compactness_se""], ""Mean Compactness vs...",0009d135ece78d
5,2843a25a,code,"# Scaling Dataʶscaler = StandardScaler()ʶscaler.fit(numerical_data)ʶ# print(scaled_data)ʶʶ# Assigning VariablesʶX = scaler.transform(numerical_data)ʶy = labelsʶʶmy_imputer = SimpleImputer()ʶpd.DataFrame(X).fillna(0)ʶX = my_imputer.fit_transform(X)ʶʶprint(""Ignore the errors, they occurred because of NaN values"")ʶprint()ʶprint(""But worry not human! The errors are fixed with Imputer >o>"")ʶprint()",0009d135ece78d
6,06dbf8cf,code,"# 3. Implementing PCA on X (green for benign; red for malignant)ʶ################################################################ʶʶ# PCAʶPCA3=PCA(n_components=2)ʶ# print(X.shape)ʶPCA3.fit(X)ʶXPCA = PCA3.transform(X)ʶ# print(XPCA.shape)ʶʶ# Plottingʶplt.figure()ʶplt.title(""PCA"")ʶplt.xlabel('X-Axis')ʶplt.ylabel('Y-Axis')ʶʶplt.plot(XPCA[y==0,0],XPCA[y==0,1],'g.')ʶplt.plot(XPCA[y==1,0],XPCA[y==1,1],'r.')ʶʶplt.show()",0009d135ece78d
7,f9893819,markdown,# Scaling Data ⚖ʶLet's scale the data so PCA can be applied,0009d135ece78d
8,ba55e576,markdown,## Testing Plots >w>ʶLet's these mystery soliving plots! :O,0009d135ece78d
9,39e937ec,markdown,"## Plotting PCA 📊ʶThus, the sun boils down to this, the PCA is hence plotted 😮",0009d135ece78d


In [ ]:
cell_dict = dict(zip(df.cell_id.values, df.cell_type.values))

In [ ]:
cell_dict

{'1c7a5a71': 'code',
 '68f71f96': 'code',
 'f0a7f5c4': 'code',
 '548f02a6': 'code',
 '627f1de4': 'code',
 '13e686e4': 'code',
 'ea19279d': 'code',
 '1bc9fdd3': 'code',
 '25404635': 'code',
 'de35a540': 'code',
 'b1222c97': 'code',
 'cdddba65': 'code',
 '8ca03cb9': 'code',
 '9cdbc1a2': 'code',
 '4ca189c7': 'code',
 '0dd79fa2': 'code',
 '127ef4eb': 'code',
 'a042dc13': 'code',
 '729ed765': 'code',
 '3a7ef855': 'code',
 '2d49db47': 'code',
 '9e634dcb': 'code',
 '1d827ca4': 'code',
 'cc93345e': 'code',
 'ba9fdcc1': 'code',
 'e0145f24': 'code',
 'ee3bf863': 'code',
 'bd408029': 'code',
 '615e7fac': 'code',
 'cdec7820': 'code',
 '03681483': 'code',
 'a2bb7871': 'code',
 '88a3d628': 'code',
 'b36223e2': 'code',
 '217112f2': 'code',
 '21118351': 'code',
 '16843639': 'code',
 'b2ed0be1': 'code',
 '00175e5a': 'markdown',
 'a196e5ee': 'markdown',
 '9c545e0d': 'markdown',
 'f39ffe12': 'markdown',
 '88a0b7e5': 'markdown',
 '5bef2452': 'markdown',
 '24387665': 'markdown',
 '5d713d48': 'markdown',
 '

In [ ]:
df_orders

,id,cell_order,n_cell_id
0,00001756c60be8,1862f0a6 448eb224 2a9e43d6 7e2f170a 038b763d 77e56113 2eefe0ef 1ae087ab 0beab1cd 8ffe0b25 9a78ab76 0d136e08 8a4c95d1 23705731 ebe125d5 aaad8355 d9dced8b 21616367 86497fe1 c3ce0945 e2c8e725 a6357f7e ff7c44ed ac301a84 0e7c906e dd0c804a 45082c89 781bbf3c 4bb2e30a bd94f005 63c26fa2 62638fba 3e5f860d bb69e88c 6b5664c7 3eebeb87 23783525 36002912 bfbde93e 8522781a 1496beaf 8ca8392c b69a4f9b 17ec3fc4 503926eb 76512d50 032e2820 a98c5d9f 06365725 8554b284 59959af5 2e1a5949 80151ab7 fcb6792d 5bf9ca51 9...,58
1,00015c83e2717b,2e94bd7a 3e99dee9 b5e286ea da4f7550 c417225b 51e3cd89 2600b4eb 75b65993 cf195f8b 25699d02 72b3201a f2c750d3 de148b56 42749e24 fbe3e811 9901472c 10377ef8 1f462e2f fceeb3e6 ceba8ae0 924c9f0f 2af2a41a 91e68f13 9216a113 63753f10 d5aee1e4 dc8a39a5 3d0a28c2 657a8804 0eea9701 a8fcc3e3 a6f8f9f1 7223cfc2 a166703b df6b3ccb da8b817a a9b266d2 41beeead 3e17f424 cd61f6d1 42f0c365 cc8d23d8 be6c9079 ad42abc1 7894c4e8 eab2b130 cc1add42 16b0d436 a3e791de 183226e4 02ef0932 03441163 f2915b9f d429a743 4d5ebd46 3...,93
2,0001bdd4021779,3fdc37be 073782ca 8ea7263c 80543cd8 38310c80 073e27e5 015d52a4 ad7679ef 7fde4f04 07c52510 0a1a7a39 0bcd3fef 58bf360b,13
3,0001daf4c2c76d,97266564 a898e555 86605076 76cc2642 ef279279 df6c939f 2476da96 00f87d0a ae93e8e6 58aadb1d d20b0094 986fd4f1 b4ff1015 9b761026 6f271c86 97c3f99b 2451daed cfa510c5 374a5179 df5f7c1f 27060ab9 4c7395ec 2d2f9893 35c2812a a6617a52 cd2cf0ad 50cc03dd a57f19d3 4843d706 09c20343 4385fc90 42241549 edaaeecd 1ed4aebf 54ddeb88 08dac4fd 3307d698 ebb2c2f7 ac2b6b85 34f430ca e54d5869 39c98d5f d6c5745e 966d9c8a 27f52431 4524758d 91a4584a 4ef89e46 82d7d8ca b6c234fb 2d0e1fda 434b8e0c bd0b9dd7 7cececae a3edb582 a...,229
4,0002115f48f982,9ec225f0 18281c6c e3b6b115 4a044c54 365fe576 a3188e54 b3f6e12d ee7655ca 84125b7a,9
...,...,...,...
139251,fffc30d5a0bc46,09727c0c ff1ea6a0 ddfef603 a01ce9b3 3ba953ee bf92a015 f4a0492a 095812e6 53125cfe aa32a700 63340e73 06d8c952 400ceb37 55fa544e d3647ac8 f8e7d47e 7102897a a3a47643 ebd32b52 53caf4d9 b314d9e3 2986b5e1 7eb7350d 681c0d9b f083e1de 704b3ee9 a8e79846 eb887c6c 4faf1ad7 6122a91f 015cff34,31
139252,fffc3b44869198,978a5137 faa48f03 28dfb12a eea2e812 64fef97c 4e0d1510 58e68f2c 8784e700 4bd5a4cf dc14bfec 2aff7603 8047ded2 ac536a5b d2ec3efb d7172db2 0c5441b4 8bb4abd2 aa168c04 b01422ce 6e67b343 40e930ff b1873cbb 76e0f2a7 233d93b9,24
139253,fffc63ff750064,5015c300 411b85d9 8238198c f4781d1d b5532930 e1f223e5 e7e67119 4aaf741d 7229cce6 a7fa3628 e4c2fa86 1f8f9d14 7a52357a e9b6265c b32dc5d2 41ccef58 6aff1ae7 29e1f39d b3c6bc16 37cbd460 79e4e69f ee9b8d31 8b54cf58 deead32c 56ee6e4c 56aa8da7,26
139254,fffcd063cda949,7e6266ad d8281fc5 d4ffcaef 3e0e4a47 21387fc8 cc229f9a baed9c8b d1bb21aa 82979992 65f95dad eba4fa9e c97e268d 7e1a6588 055e0d2e c090bd72 0bf4ab17 26374693 5098ff9a ef42f19d 28f8bc15 b47bacee 0fad41c8 4f671884 4b254efa 74802b2f 7c33c5b2 7168afec 9a2cdb85 fb7456dd 1fd0781b ff949d21 a16411ac 7a1b4718 777b3fc0 f90ee056 83dce89a,36


In [ ]:
def to_rank(df_orders):
  ids = []
  cell_ids = []
  ranks = []
  for row in df_orders.itertuples():
    cells = row.cell_order.split()
    ids.extend([row.id] * len(cells))
    cell_ids.extend(cells)
    ranks.extend(list(range(len(cells))))
  df_rank = pd.DataFrame({
    'id': ids,
    'cell_id': cell_ids,
    'rank': ranks,
  })
  return df_rank

In [ ]:
df_rank = to_rank(df_orders)
df_rank

,id,cell_id,rank
0,00001756c60be8,1862f0a6,0
1,00001756c60be8,448eb224,1
2,00001756c60be8,2a9e43d6,2
3,00001756c60be8,7e2f170a,3
4,00001756c60be8,038b763d,4
...,...,...,...
6370641,fffe1d764579d5,5369934b,67
6370642,fffe1d764579d5,ac9db030,68
6370643,fffe1d764579d5,a8ffc8b4,69
6370644,fffe1d764579d5,70455170,70


In [ ]:
df_orders.cell_order.apply(lambda x: x.split()).apply(pd.Series)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004
0,1862f0a6,448eb224,2a9e43d6,7e2f170a,038b763d,77e56113,2eefe0ef,1ae087ab,0beab1cd,8ffe0b25,9a78ab76,0d136e08,8a4c95d1,23705731,ebe125d5,aaad8355,d9dced8b,21616367,86497fe1,c3ce0945,e2c8e725,a6357f7e,ff7c44ed,ac301a84,0e7c906e,dd0c804a,45082c89,781bbf3c,4bb2e30a,bd94f005,63c26fa2,62638fba,3e5f860d,bb69e88c,6b5664c7,3eebeb87,23783525,36002912,bfbde93e,8522781a,1496beaf,8ca8392c,b69a4f9b,17ec3fc4,503926eb,76512d50,032e2820,a98c5d9f,06365725,8554b284,59959af5,2e1a5949,80151ab7,fcb6792d,5bf9ca51,915643b3,f5504853,9f50dca0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [ ]:
df = df.merge(df_rank, on=['id', 'cell_id'], how='inner')

In [ ]:
df = df.sort_values(['id', 'rank'])

In [ ]:
ids = list(set(df.id))

In [ ]:
df[df.id==ids[102]]

,index,cell_id,cell_type,source,id,input_ids,n_words,rank,inpu_ids2,input_ids2
2911522,31,7bb92ebe,markdown,"Hey kagglers,ʶʶThis is my first competition in kaggle, I would like to share my approach on the Cassava leaf disease competition.ʶMy approach involoves:ʶ* Using keras libraryʶ* Using GPU to train the modelʶ* Using albumentations to augment the dataset to prevent overfittingʶ* Using the StratifiedKFold as the dataset is skewed.ʶʶThis notebook is for beginner who would like to get started with this competitionʶʶReferences:ʶ1. Approaching (Almost) Any Machine Learning Problem - Book by Abhishek...",43bd04946376da,"[1, 5388, 7478, 94989, 268, 261, 128001, 128001, 329, 269, 312, 362, 2027, 267, 4219, 89061, 261, 273, 338, 334, 264, 752, 312, 1218, 277, 262, 18961, 16925, 6712, 1682, 2027, 260, 128001, 573, 1218, 267, 90974, 415, 268, 294, 128001, 1124, 2891, 39170, 1628, 2634, 128001, 1124, 2891, 16265, 264, 2184, 262, 1040, 128001, 1124, 2891, 1898, 91547, 268, 264, 24906, 262, 12438, 264, 1843, 360, 25335, 128001, 1124, 2891, 262, 38251, 13329, 51708, 1371, 283, 262, 12438, 269, 32147, 260, 128001, 12...",208,0,"[1, 5388, 7478, 94989, 268, 261, 128001, 128001, 329, 269, 312, 362, 2027, 267, 4219, 89061, 261, 273, 338, 334, 264, 752, 312, 1218, 277, 262, 18961, 16925, 6712, 1682, 2027, 260, 128001, 573, 1218, 267, 90974, 415, 268, 294, 128001, 1124, 2891, 39170, 1628, 2634, 128001, 1124, 2891, 16265, 264, 2184, 262, 1040, 128001, 1124, 2891, 1898, 91547, 268, 264, 24906, 262, 12438, 264, 1843, 360, 25335, 128001, 1124, 2891, 262, 38251, 13329, 51708, 1371, 283, 262, 12438, 269, 32147, 260, 128001, 12...","[1, 5388, 7478, 94989, 268, 261, 128001, 128001, 329, 269, 312, 362, 2027, 267, 4219, 89061, 261, 273, 338, 334, 264, 752, 312, 1218, 277, 262, 18961, 16925, 6712, 1682, 2027, 260, 128001, 573, 1218, 267, 90974, 415, 268, 294, 128001, 1124, 2891, 39170, 1628, 2634, 128001, 1124, 2891, 16265, 264, 2184, 262, 1040, 128001, 1124, 2891, 1898, 91547, 268, 264, 24906, 262, 12438, 264, 1843, 360, 25335, 128001, 1124, 2891, 262, 38251, 13329, 51708, 1371, 283, 262, 12438, 269, 32147, 260, 128001, 12..."
2911521,30,9464c9af,markdown,# **Importing libraries**,43bd04946376da,"[1, 953, 1124, 1225, 60098, 510, 7296, 1225, 1225, 2]",10,1,"[1, 953, 1124, 1225, 60098, 510, 7296, 1225, 1225, 2]","[1, 953, 1124, 1225, 60098, 510, 7296, 1225, 1225, 2]"
2911491,0,3f1ad88e,code,"import pandas as pdʶimport numpy as npʶimport matplotlib.pyplot as pltʶimport os ʶfrom sklearn.model_selection import StratifiedKFoldʶimport seaborn as snsʶimport jsonʶimport cv2ʶʶimport tensorflow as tfʶfrom tensorflow.keras import models, layers,Sequential,regularizersʶfrom tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping, ModelCheckpointʶfrom keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropoutʶfrom tensorflow.keras.models impor...",43bd04946376da,"[1, 6306, 67927, 283, 845, 407, 128001, 6306, 36221, 11751, 283, 76767, 128001, 6306, 8358, 33918, 14434, 260, 11751, 33918, 283, 28944, 297, 128001, 6306, 2673, 268, 128001, 292, 33566, 29274, 260, 19928, 616, 57907, 6306, 38251, 13329, 51708, 1371, 128001, 6306, 2164, 6107, 283, 41339, 268, 128001, 6306, 74883, 128001, 6306, 37821, 445, 128001, 128001, 6306, 55919, 14068, 283, 78191, 128001, 292, 55919, 14068, 260, 8285, 1628, 6306, 1836, 261, 4974, 261, 430, 87386, 46580, 261, 34020, 4322...",242,2,"[1, 6306, 67927, 283, 845, 407, 128001, 6306, 36221, 11751, 283, 76767, 128001, 6306, 8358, 33918, 14434, 260, 11751, 33918, 283, 28944, 297, 128001, 6306, 2673, 268, 128001, 292, 33566, 29274, 260, 19928, 616, 57907, 6306, 38251, 13329, 51708, 1371, 128001, 6306, 2164, 6107, 283, 41339, 268, 128001, 6306, 74883, 128001, 6306, 37821, 445, 128001, 128001, 6306, 55919, 14068, 283, 78191, 128001, 292, 55919, 14068, 260, 8285, 1628, 6306, 1836, 261, 4974, 261, 430, 87386, 46580, 261, 34020, 4322...","[1, 6306, 67927, 283, 845, 407, 128001, 6306, 36221, 11751, 283, 767

In [ ]:
df_test.head(10)

,cell_id,cell_type,source,id
0,ddfd239c,code,"import numpy as np # linear algebraʶimport pandas as pd # data processing,ʶimport matplotlib.pyplot as pltʶfrom sklearn.decomposition import PCAʶfrom sklearn.preprocessing import StandardScalerʶfrom sklearn.preprocessing import scaleʶfrom sklearn.impute import SimpleImputerʶʶʶimport osʶfor dirname, _, filenames in os.walk('/kaggle/input'):ʶ for filename in filenames:ʶ print(os.path.join(dirname, filename))",0009d135ece78d
1,c6cd22db,code,df = pd.read_csv('/kaggle/input/breast-cancer-wisconsin-data/data.csv')ʶdf,0009d135ece78d
2,1372ae9b,code,"numerical_data = df.loc[:, ~df.columns.isin(['id', ""diagnosis""])]ʶʶlabels = df[""diagnosis""].factorize(['B','M'])[0]ʶʶheader_labels = pd.DataFrame(data=labels, columns=[""diagnosis""])",0009d135ece78d
3,90ed07ab,code,"def comparison_plot_maker(data_1, data_2, name, column_name_1, column_name_2):ʶ # Scaling Data for testingʶ # data_1 = scale(data_1)ʶ # data_2 = scale(data_2)ʶʶ range = np.random.randn(len(data_1))ʶ plt.scatter(range, data_1, label=column_name_1, color='orange')ʶ plt.scatter(range, data_2, label=column_name_2, color='green')ʶ plt.title(name)ʶ plt.xlabel('X-Axis')ʶ plt.ylabel('Y-Axis')ʶ plt.legend()ʶ plt.show()ʶ",0009d135ece78d
4,7f388a41,code,"# Ploting data with different columnsʶ#####################################ʶcomparison_plot_maker(numerical_data[""radius_mean""], numerical_data[""radius_worst""], ""Mean Radius vs Worst Radius"", ""Mean Radius"", ""Worst Radius"")ʶcomparison_plot_maker(numerical_data[""perimeter_se""], numerical_data[""perimeter_worst""], ""S.D Perimeter vs Worst Perimeter"", ""S.D Perimeter"", ""Worst Perimeter"")ʶcomparison_plot_maker(numerical_data[""compactness_mean""], numerical_data[""compactness_se""], ""Mean Compactness vs...",0009d135ece78d
5,2843a25a,code,"# Scaling Dataʶscaler = StandardScaler()ʶscaler.fit(numerical_data)ʶ# print(scaled_data)ʶʶ# Assigning VariablesʶX = scaler.transform(numerical_data)ʶy = labelsʶʶmy_imputer = SimpleImputer()ʶpd.DataFrame(X).fillna(0)ʶX = my_imputer.fit_transform(X)ʶʶprint(""Ignore the errors, they occurred because of NaN values"")ʶprint()ʶprint(""But worry not human! The errors are fixed with Imputer >o>"")ʶprint()",0009d135ece78d
6,06dbf8cf,code,"# 3. Implementing PCA on X (green for benign; red for malignant)ʶ################################################################ʶʶ# PCAʶPCA3=PCA(n_components=2)ʶ# print(X.shape)ʶPCA3.fit(X)ʶXPCA = PCA3.transform(X)ʶ# print(XPCA.shape)ʶʶ# Plottingʶplt.figure()ʶplt.title(""PCA"")ʶplt.xlabel('X-Axis')ʶplt.ylabel('Y-Axis')ʶʶplt.plot(XPCA[y==0,0],XPCA[y==0,1],'g.')ʶplt.plot(XPCA[y==1,0],XPCA[y==1,1],'r.')ʶʶplt.show()",0009d135ece78d
7,f9893819,markdown,# Scaling Data ⚖ʶLet's scale the data so PCA can be applied,0009d135ece78d
8,ba55e576,markdown,## Testing Plots >w>ʶLet's these mystery soliving plots! :O,0009d135ece78d
9,39e937ec,markdown,"## Plotting PCA 📊ʶThus, the sun boils down to this, the PCA is hence plotted 😮",0009d135ece78d
